# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.environment import Environment

In [2]:
from azureml.core import Workspace, Experiment

In [3]:
# Environment(name="myenv")

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
ws = Workspace.from_config()

In [5]:
env = Environment.get(workspace = ws, name = "AzureML-Tutorial")

In [6]:

# env = Environment.get(workspace = ws, name = "AzureML-SKLearn-0.24-ubuntu18.04-py37-cpu")

In [7]:
experiment_name = 'CapstoneProject_HyperDrive'

experiment=Experiment(ws, experiment_name)

In [8]:
print(ws.name)

quick-starts-ws-155764


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [9]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "UdacityProject"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [10]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- scikit-learn
- pandas
- pip:
  - azureml-defaults

Writing conda_dependencies.yml


In [11]:
from azureml.core import Environment

sklearn_env = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')


from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

In [14]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=10)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        'C': uniform(0.001, 1.0), # Regularization
        'max_iter': choice(5, 20, 100, 200) # Max number of iterations (aka epochs)
    }
)


hyperdrive_run_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=param_sampling,
                             policy=early_termination_policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20,
                             max_concurrent_runs=4)
#                                         

In [15]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [16]:
from azureml.widgets import RunDetails
RunDetails(hyperdrive_run).show()
# hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [22]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n learning rate:',parameter_values[3])
print('\n Accuracy:',best_run_metrics['Accuracy'])



Best Run Id:  HD_2ac88e36-b1b9-4767-9792-4c660f80f34f_1

 learning rate: 100

 Accuracy: 0.8833333333333333


In [23]:
parameter_values

['--C', '0.46663318997428704', '--max_iter', '100']

In [24]:
best_run_metrics

{'Regularization Strength:': 0.46663318997428704,
 'Max iterations:': 100,
 'Accuracy': 0.8833333333333333}

In [25]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
CapstoneProject_HyperDrive,HD_2ac88e36-b1b9-4767-9792-4c660f80f34f_1,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service